In [2]:
# ProstateX dataset preprocess
import os
import shutil

dst_root_path = "/path/to/prostate158_PROSTATEx_dataset/PROSTATEx"  # save path
os.makedirs(dst_root_path, exist_ok=True)
dst_image_path = os.path.join(dst_root_path, "images")
os.makedirs(dst_image_path, exist_ok=True)
dst_whole_path = os.path.join(dst_root_path, "whole")
os.makedirs(dst_whole_path, exist_ok=True)
dst_tz_path = os.path.join(dst_root_path, "tz")
os.makedirs(dst_tz_path, exist_ok=True)
dst_pz_path = os.path.join(dst_root_path, "pz")
os.makedirs(dst_pz_path, exist_ok=True)

images_root_path = "/path/to/PROSTATEx_masks/Files/prostate/Images"
image_path_list = os.listdir(images_root_path)
for image_path in image_path_list:
    new_image_path = image_path.split("_")[0] + ".nii.gz"
    shutil.copyfile(os.path.join(images_root_path, image_path), os.path.join(dst_image_path, new_image_path))

whole_root_path = "/path/to/PROSTATEx_masks/Files/prostate/mask_prostate"
whole_path_list = os.listdir(whole_root_path)
for whole_path in whole_path_list:
    id = whole_path.split(".")[0].split("-")[1]
    format_id = '{:04}'.format(int(id))
    new_whole_path = "ProstateX-" + format_id + ".nii.gz"
    shutil.copyfile(os.path.join(whole_root_path, whole_path), os.path.join(dst_whole_path, new_whole_path))

tz_root_path = "/path/to/PROSTATEx_masks/Files/prostate/mask_tz"
tz_path_list = os.listdir(tz_root_path)
for tz_path in tz_path_list:
    id = tz_path.split(".")[0].split("-")[1].split("_")[0]
    format_id = '{:04}'.format(int(id))
    new_tz_path = "ProstateX-" + format_id + ".nii.gz"
    shutil.copyfile(os.path.join(tz_root_path, tz_path), os.path.join(dst_tz_path, new_tz_path))

pz_root_path = "/path/to/PROSTATEx_masks/Files/prostate/mask_pz"
pz_path_list = os.listdir(pz_root_path)
for pz_path in pz_path_list:
    id = pz_path.split(".")[0].split("-")[1].split("_")[0]
    format_id = '{:04}'.format(int(id))
    new_pz_path = "ProstateX-" + format_id + ".nii.gz"
    shutil.copyfile(os.path.join(pz_root_path, pz_path), os.path.join(dst_pz_path, new_pz_path))

In [ ]:
import os
import numpy as np
import SimpleITK as sitk


dst_anatomy_path = "/path/to/prostate158_PROSTATEx_dataset/PROSTATEx/anatomy"
tz_path = "/path/to/prostate158_PROSTATEx_dataset/PROSTATEx/tz/"
pz_path = "/path/to/prostate158_PROSTATEx_dataset/PROSTATEx/pz/"

id_path_list = os.listdir(tz_path)
for id in id_path_list:
    tz = sitk.ReadImage(os.path.join(tz_path, id))
    tz_array = sitk.GetArrayFromImage(tz)
    pz = sitk.ReadImage(os.path.join(pz_path, id))
    pz_array = sitk.GetArrayFromImage(pz)

    mask_tz = tz_array == 1
    mask_pz = pz_array == 1

    anatomy = np.zeros_like(tz_array)
    anatomy[mask_tz] = 1
    anatomy[mask_pz] = 2

    anatomy = sitk.GetImageFromArray(anatomy)
    anatomy.CopyInformation(tz)

    sitk.WriteImage(anatomy, os.path.join(dst_anatomy_path, id))

In [4]:
# Prostate158 dataset preprocess
import os
import shutil

root_path = "/path/to/prostate158/train"
dst_images_path = "/path/to/prostate158_PROSTATEx_dataset/prostate158/images/"
dst_anatomy_path = "/path/to/prostate158_PROSTATEx_dataset/prostate158/anatomy/"

id = 0
for dir in os.listdir(root_path):
    format_id = '{:04}'.format(int(id))
    id += 1
    dir_path = os.path.join(root_path, dir)
    for file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file)
        if file == "t2.nii.gz":
            shutil.copyfile(file_path, os.path.join(dst_images_path, "prostate158-" + format_id + ".nii.gz"))
        elif file == "t2_anatomy_reader1.nii.gz":
            shutil.copyfile(file_path, os.path.join(dst_anatomy_path, "prostate158-" + format_id + ".nii.gz"))

In [9]:
import os
import numpy as np
import SimpleITK as sitk


dst_whole_path = "/path/to/prostate158_PROSTATEx_dataset/prostate158/whole/"
dst_tz_path = "/path/to/prostate158_PROSTATEx_dataset/prostate158/tz/"
dst_pz_path = "/path/to/prostate158_PROSTATEx_dataset/prostate158/pz/"
anatomy_path = "/path/to/prostate158/prostate158_PROSTATEx_dataset/anatomy/"

anatomy_path_list = os.listdir(anatomy_path)
for anatomy in anatomy_path_list:
    img = sitk.ReadImage(os.path.join(anatomy_path, anatomy))
    img_array = sitk.GetArrayFromImage(img)

    mask_tz = img_array == 1
    mask_pz = img_array == 2
    mask_whole = (img_array == 1) | (img_array == 2)
    tz = np.zeros_like(img_array)
    tz[mask_tz] = 1
    pz = np.zeros_like(img_array)
    pz[mask_pz] = 1
    whole = np.zeros_like(img_array)
    whole[mask_whole] = 1

    tz_image = sitk.GetImageFromArray(tz)
    pz_image = sitk.GetImageFromArray(pz)
    whole_image = sitk.GetImageFromArray(whole)

    tz_image.CopyInformation(img)
    pz_image.CopyInformation(img)
    whole_image.CopyInformation(img)

    sitk.WriteImage(tz_image, os.path.join(dst_tz_path, anatomy))
    sitk.WriteImage(pz_image, os.path.join(dst_pz_path, anatomy))
    sitk.WriteImage(whole_image, os.path.join(dst_whole_path, anatomy))

In [ ]:
# Split into training, validation, and test sets
import pandas as pd

csv_path = "/path/to/prostate158/valid.csv"
df = pd.read_csv(csv_path)
val_id_data = df["ID"]
val_id_list = val_id_data.tolist()
val_id_list = ['{:03}'.format(int(val_id)) for val_id in val_id_data]
val_id_list

In [ ]:
import os
import shutil

train_val_path = "/path/to/prostate158/train"
test_path = "/path/to/prostate158/test"
dst_train_images_path = "/path/to/prostate158_PROSTATEx_dataset/train/images"
dst_train_anatomy_path = "/path/to/prostate158_PROSTATEx_dataset/train/anatomy"
dst_val_images_path = "/path/to/prostate158_PROSTATEx_dataset/vaild/images"
dst_val_anatomy_path = "/path/to/prostate158_PROSTATEx_dataset/vaild/anatomy"
dst_test_images_path = "/path/to/prostate158_PROSTATEx_dataset/test/images"
dst_test_anatomy_path = "/path/to/prostate158_PROSTATEx_dataset/test/anatomy"
os.makedirs(dst_train_images_path, exist_ok=True)
os.makedirs(dst_train_anatomy_path, exist_ok=True)
os.makedirs(dst_val_images_path, exist_ok=True)
os.makedirs(dst_val_anatomy_path, exist_ok=True)
os.makedirs(dst_test_images_path, exist_ok=True)
os.makedirs(dst_test_anatomy_path, exist_ok=True)

dst_images_path = ""
dst_anatomy_path = ""
for data_dir in os.listdir(train_val_path):
    dir_path = os.path.join(train_val_path, data_dir)
    format_id = '{:04}'.format(int(data_dir))
    if data_dir in val_id_list:
        dst_images_path = dst_val_images_path
        dst_anatomy_path = dst_val_anatomy_path
    else:
        dst_images_path = dst_train_images_path
        dst_anatomy_path = dst_train_anatomy_path

    for file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file)
        if file == "t2.nii.gz":
            shutil.copyfile(file_path, os.path.join(dst_images_path, "prostate158-" + format_id + ".nii.gz"))
        elif file == "t2_anatomy_reader1.nii.gz":
            shutil.copyfile(file_path, os.path.join(dst_anatomy_path, "prostate158-" + format_id + ".nii.gz"))


for data_dir in os.listdir(test_path):
    dir_path = os.path.join(test_path, data_dir)

    format_id = '{:04}'.format(int(data_dir))

    for file in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file)
        if file == "t2.nii.gz":
            shutil.copyfile(file_path, os.path.join(dst_test_images_path, "prostate158-" + format_id + ".nii.gz"))
        elif file == "t2_anatomy_reader1.nii.gz":
            shutil.copyfile(file_path, os.path.join(dst_test_anatomy_path, "prostate158-" + format_id + ".nii.gz"))

In [ ]:
PROSTATEx_dir = "/path/to/prostate158_PROSTATEx_dataset/PROSTATEx"
data_name_list = os.listdir(os.path.join(PROSTATEx_dir, "images"))
data_name_list = sorted(data_name_list)

for i in range(len(data_name_list)):
    if i <= 153:
        dst_dir = "/path/to/prostate158_PROSTATEx_dataset/train"
    elif i <= 178:
        dst_dir = "/path/to/prostate158_PROSTATEx_dataset/vaild"
    else:
        dst_dir = "/path/to/prostate158_PROSTATEx_dataset/test"

    img_path = os.path.join(PROSTATEx_dir, "images", data_name_list[i])
    anatomy_path = os.path.join(PROSTATEx_dir, "anatomy", data_name_list[i])
    shutil.copyfile(img_path, os.path.join(dst_dir, "images", data_name_list[i]))
    shutil.copyfile(anatomy_path, os.path.join(dst_dir, "anatomy", data_name_list[i]))


In [ ]:
# train/val/test data2csv
import csv
import os

train_dir = "/path/to/prostate158_PROSTATEx_dataset/train"
train_images_dir = os.path.join(train_dir, "images")
train_anatomy_dir = os.path.join(train_dir, "anatomy")
val_dir = "/path/to/prostate158_PROSTATEx_dataset/vaild"
val_images_dir = os.path.join(val_dir, "images")
val_anatmy_dir = os.path.join(val_dir, "anatomy")
test_dir = "/path/to/prostate158_PROSTATEx_dataset/test"
test_images_dir = os.path.join(test_dir, "images")
test_anatomy_dir = os.path.join(test_dir, "anatomy")


train_id_list = sorted(os.listdir(train_images_dir))
train_data_list = []
head = ['ID', 't2', 'anatomy']
train_data_list.append(head)

for train_id in train_id_list:
    t2_path = "train/images/" + train_id
    anatomy_path = "train/anatomy/" + train_id
    case_list = [train_id.split(".")[0], t2_path, anatomy_path]
    train_data_list.append(case_list)

with open('/path/to/prostate_PROSTATEx_train.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(train_data_list)

val_id_list = sorted(os.listdir(val_images_dir))
val_data_list = []
head = ['ID', 't2', 'anatomy']
val_data_list.append(head)

for val_id in val_id_list:
    t2_path = "vaild/images/" + val_id
    anatomy_path = "vaild/anatomy/" + val_id
    case_list = [val_id.split(".")[0], t2_path, anatomy_path]
    val_data_list.append(case_list)

with open('/path/to/prostate_PROSTATEx_vaild.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(val_data_list)

test_id_list = sorted(os.listdir(test_images_dir))
test_data_list = []
head = ['ID', 't2', 'anatomy']
test_data_list.append(head)

for test_id in test_id_list:
    t2_path = "test/images/" + test_id
    anatomy_path = "test/anatomy/" + test_id
    case_list = [test_id.split(".")[0], t2_path, anatomy_path]
    test_data_list.append(case_list)

with open('/path/to/prostate_PROSTATEx_test.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(test_data_list)
